In [12]:
import gradio as gr
from ultralytics import YOLO
from PIL import Image
import easyocr

# Cargar modelo entrenado
model = YOLO("/home/gnz/GitHub/yolo11_container/runs/detect/train/weights/best.pt")

# Inicializar OCR (idioma inglés, podés agregar otros ej. ['en','es'])
reader = easyocr.Reader(['en','es'])


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [ ]:
# Función de predicción con cropeo + OCR
def predict_with_ocr(image):
    results = model.predict(image, conf=0.25, verbose=False)

    # Imagen anotada con bounding boxes
    img_with_boxes = results[0].plot()

    crops_with_text = []
    for i, box in enumerate(results[0].boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        crop = image[y1:y2, x1:x2]

        # OCR sobre el recorte
        ocr_result = reader.readtext(crop)
        if ocr_result:
            text = " ".join([res[1] for res in ocr_result])  # concatenar resultados
        else:
            text = "(sin texto detectado)"

        # Guardar crop y texto asociado
        crops_with_text.append((Image.fromarray(crop), text))

    return img_with_boxes, crops_with_text

# Interfaz Gradio
with gr.Blocks() as demo:
    gr.Markdown("## 🚢 Container OCR Detector con YOLO + EasyOCR")
    gr.Markdown("Subí una imagen de un contenedor. El modelo detecta los códigos, recorta las regiones y aplica OCR para mostrar el texto.")

    with gr.Row():
        inp = gr.Image(type="numpy", label="Imagen de entrada")

    with gr.Row():
        out_img = gr.Image(type="numpy", label="Resultado con bounding boxes")

    with gr.Row():
        out_gallery = gr.Gallery(label="Crops + OCR", columns=2, height="auto")

    btn = gr.Button("Detectar, recortar y leer texto")
    btn.click(fn=predict_with_ocr, inputs=inp, outputs=[out_img, out_gallery])

demo.launch(server_name="0.0.0.0", server_port=7866)

* Running on local URL:  http://0.0.0.0:7866
* To create a public link, set `share=True` in `launch()`.


/home/gnz/GitHub/yolo11_container/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/gnz/GitHub/yolo11_container/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/gnz/GitHub/yolo11_container/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/gnz/GitHub/yolo11_container/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_m